## Import libraries

In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

## Create a directory to store data

In [2]:
isExist = os.path.exists('./snotel_data1')
if isExist == True:
    exit
else:
    os.mkdir('./snotel_data1')

In [3]:
%%bash
pwd

/bsuscratch/stanleyakor/uppercolorado/ml_frameworks/evaluation/DOWNLOAD_SNOTEL


## Read the file that contains information about SNOTEL sites

In [4]:
SNOTEL_info = pd.read_csv('./NRCS_SNOTEL_Locations_noAlaska.csv')

In [5]:
set(SNOTEL_info['State'])

{'AZ', 'CA', 'CO', 'ID', 'MT', 'NM', 'NV', 'OR', 'SD', 'UT', 'WA', 'WY'}

In [6]:
SNOTEL_info.head(3)

,Station ID,State,Network,Station Name,Latitude,Longitude,Elevation,HUC2 (2-digit),HUC2 Name,HUC4 (4-digit),...,HUC6 Name,HUC8 (8-digit),HUC8 Name,HUC10 (10-digit),HUC10 Name,HUC12 (12-digit),HUC12 Name,Report Time Zone (Offset from GMT),Start Date,End Date
0,301,CA,SNOTEL,Adin Mtn,41.23583,-120.79192,6190,18,California Region,1802,...,Upper Sacramento,18020002,Upper Pit,1802000214,Cottonwood Creek-Ash Creek,1.800000e+11,Cottonwood Creek,-8,10/1/1983,1/1/2100
1,907,UT,SNOTEL,Agua Canyon,37.52217,-112.27118,8900,16,Great Basin Region,1603,...,Escalante Desert-Sevier Lake,16030002,East Fork Sevier,1603000203,Upper East Fork Sevier River,1.600000e+11,East Fork Sevier River Headwaters,-8,10/1/1994,1/1/2100
2,916,MT,SNOTEL,Albro Lake,45.59723,-111.95902,8300,10,Missouri Region,1002,...,Missouri Headwaters,10020005,Jefferson,1002000507,Willow Creek,1.000000e+11,South Willow Creek,-8,9/1/1996,1/1/2100


## Define parameters

In [7]:
StartDate = '2014'
EndDate = '2015'
State = 'CO'
Names = list(SNOTEL_info[SNOTEL_info['State']==State]['Station Name'].values)
IDs = list(SNOTEL_info[SNOTEL_info['State']==State]['Station ID'].values)
Lat = list(SNOTEL_info[SNOTEL_info['State'] ==State]['Latitude'].values)
Lon = list(SNOTEL_info[SNOTEL_info['State'] ==State]['Longitude'].values)
Elevation = list(SNOTEL_info[SNOTEL_info['State'] ==State]['Elevation'].values)

In [8]:
# SNOTEL = pd.DataFrame({'Name':Names, 'ID':IDs, 'Latitude':Lat, 'Longitude':Lon})
# SNOTEL['State'] = State

#### For CALIFORNIA, turn on the following lines:
### It seems the site name changed from "Squaw Valley G.C." to "Palisades Tahoe "
# SNOTEL.loc[SNOTEL['ID']==784, 'Name']="Palisades Tahoe"

# ### For UTAH, turn on the following lines:
# ## There is two space between the terms on the website (if opening the url)
# SNOTEL.loc[SNOTEL['ID']==763, 'Name']="Smith  Morehouse"
# print(SNOTEL[SNOTEL['ID']==763]['Name']=='Smith  Morehouse')

SNOTEL = pd.DataFrame({'Name':Names, 'ID':IDs, 'Latitude':Lat, 'Longitude':Lon, 'Elevation':Elevation})
SNOTEL['State'] = State

SNOTEL

,Name,ID,Latitude,Longitude,Elevation,State
0,Apishapa,303,37.33062,-105.06749,10000,CO
1,Arapaho Ridge,1030,40.35098,-106.38142,10960,CO
2,Bear Lake,322,40.31118,-105.64483,9500,CO
3,Bear River,1061,40.06153,-107.00955,9080,CO
4,Beartown,327,37.71409,-107.51212,11600,CO
...,...,...,...,...,...,...
110,Wild Basin,1042,40.20105,-105.60248,9560,CO
111,Willow Creek Pass,869,40.34703,-106.09433,9540,CO
112,Willow Park,870,40.43254,-105.73337,10700,CO
113,Wolf Creek Summit,874,37.47922,-106.80170,11000,CO


In [9]:
# Some SNOTEL sites in the Boise Rockies

Required = SNOTEL[(40.317627 >= SNOTEL['Latitude']) & (SNOTEL['Latitude'] >= 36.810326) & (-105.09583 >= SNOTEL['Longitude']) & (SNOTEL['Longitude'] >= -109.0985)]
Required



,Name,ID,Latitude,Longitude,Elevation,State
2,Bear Lake,322,40.31118,-105.64483,9500,CO
3,Bear River,1061,40.06153,-107.00955,9080,CO
4,Beartown,327,37.71409,-107.51212,11600,CO
5,Beaver Ck Village,1041,39.59917,-106.51142,8500,CO
6,Berthoud Summit,335,39.80392,-105.77789,11300,CO
...,...,...,...,...,...,...
107,Wager Gulch,1188,37.88248,-107.36430,11100,CO
108,Weminuche Creek,1160,37.51976,-107.32152,10740,CO
109,Whiskey Ck,857,37.21411,-105.12245,10220,CO
110,Wild Basin,1042,40.20105,-105.60248,9560,CO


In [10]:

# # subset to Bogus Basin, return a dataframe with just Bogus basin
# bogus_basin = Required.iloc[3].to_frame().T
# bogus_basin

## Retrieve data by calling getData.py script
The python script retrieves the following variable:
- PREC: PRECIPITATION ACCUMULATION
- PRCP: PRECIPITATION INCREMENT
- PRCPSA: PRECIPITATION INCREMENT – SNOW-ADJUSTED
- WTEQ: SNOW WATER EQUIVALENT
- SNRR: SNOW RAIN RATIO
If you need other variables, please visit https://wcc.sc.egov.usda.gov/reportGenerator/ and https://www.wcc.nrcs.usda.gov/web_service/AWDB_Web_Service_Reference.htm, and make corresponding changes the python script.  

The script also re-calculates SNRR based on snow-adjusted precipitation data because the original SNRR is calculated using unadjusted precipitation. 


In [ ]:
# for i in SNOTEL.index:
#     !python collect_snotel2.py "{SNOTEL['Name'][i]}" {SNOTEL['ID'][i]} {SNOTEL['State'][i]} {StartDate} {EndDate}

for i in Required.index:
    !python download_snotel.py "{Required['Name'][i]}" {Required['ID'][i]} {Required['State'][i]} {StartDate} {EndDate}

## Move CSV files to the directory that was created earlier

In [ ]:
!mv ./df_*.csv ./snotel_data1/